# comercio_ext_indices.tb_referencia_isic_cuci
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/ISIC_CUCI_delta`
## 📌 Descrição do arquivo

Tabela de **classificação econômica internacional**, que relaciona **atividades econômicas (ISIC)** com **grupos de produtos (CUCI)**.

Esse arquivo **não contém valores, volumes ou operações comerciais**.  
Ele existe exclusivamente para **interpretação econômica e setorial**.

Fonte: MDIC / classificações internacionais (ISIC / CUCI)  
Tipo: Tabela de referência (dimensão / bridge)

| Coluna          | Descrição técnica    | Interpretação humana             |
| --------------- | -------------------- | -------------------------------- |
| `CO_ISIC_SECAO` | Código da seção ISIC | Macroatividade econômica         |
| `NO_ISIC_SECAO` | Nome da seção ISIC   | Descrição da atividade econômica |
| `CO_CUCI_GRUPO` | Código do grupo CUCI | Grupo internacional de produtos  |
| `NO_CUCI_GRUPO` | Nome do grupo CUCI   | Tipo de bem comercializado       |

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:

val bronzePath =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/ISIC_CUCI_delta/"   
val silverPath =     "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_classificacao_isic_cuci/"   
val silverTable =    "tb_classificacao_isic_cuci"
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)



In [0]:


val silverSchema = StructType(
    Seq (
      StructField ( "CO_ISIC_SECAO" , StringType ,  nullable = false ) ,
      StructField ( "NO_ISIC_SECAO" , StringType ,  nullable = false ) ,
      StructField ( "CO_CUCI_GRUPO" , StringType ,  nullable = false ) ,
      StructField ( "NO_CUCI_GRUPO" , StringType ,  nullable = false ) ,
      StructField ( "TS_REF" , TimestampType ,  nullable = false ) ,
      StructField ( "NM_ORIGEM" , StringType ,  nullable = false ) 
    )
  )


## Extração
> #### **saprk.read**

In [0]:
val dfBronzeRaw = spark . read . format ( "delta" ) . load ( bronzePath )

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_ISIC_SECAO", trim(upper(col("CO_ISIC_SECAO").cast(StringType))))
  .withColumn("NO_ISIC_SECAO", col("NO_ISIC_SECAO").cast(StringType))
  .withColumn("CO_CUCI_GRUPO", trim(upper(col("CO_CUCI_GRUPO").cast(StringType))))
  .withColumn("NO_CUCI_GRUPO", col("NO_CUCI_GRUPO").cast(StringType))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/ISIC_CUCI_delta"))

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:

val dfValid = dfWithDefaults . filter ( 
    col("CO_ISIC_SECAO").isNotNull && 
    col("CO_CUCI_GRUPO").isNotNull
  )



In [0]:
val dfDedup = dfValid . dropDuplicates (
    Seq (
        "CO_ISIC_SECAO","CO_CUCI_GRUPO"
    )
  )

In [0]:
val dfSilver = spark.createDataFrame(dfDedup.rdd, silverSchema.fieldNames.foldLeft(dfDedup.schema)((schema, name) => schema))

##Carga
> #### **overwrite**

In [0]:
spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

val silverExists = fs.exists(new Path(silverPath))

if (!silverExists) {
  dfSilver
    .write
    .format("delta")
    .mode("overwrite")
    .save(silverPath)
}